# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921722566109                   -0.69    5.9         
  2   -7.926164715375       -2.35       -1.22    1.0    213ms
  3   -7.926836427827       -3.17       -2.37    1.6    230ms
  4   -7.926861482573       -4.60       -3.01    2.9    281ms
  5   -7.926861636249       -6.81       -3.35    1.8    218ms
  6   -7.926861667018       -7.51       -3.73    1.8    218ms
  7   -7.926861680477       -7.87       -4.35    1.2    202ms
  8   -7.926861681808       -8.88       -4.95    2.1    254ms
  9   -7.926861681859      -10.29       -5.24    1.8    217ms
 10   -7.926861681871      -10.92       -5.91    1.6    216ms
 11   -7.926861681873      -11.95       -7.05    2.1    246ms
 12   -7.926861681873      -13.56       -7.35    3.0    266ms
 13   -7.926861681873      -15.05       -8.56    1.2    204ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921691130595                   -0.69    5.8         
  2   -7.926164715151       -2.35       -1.22    1.0    212ms
  3   -7.926836463818       -3.17       -2.37    1.6    231ms
  4   -7.926861487798       -4.60       -3.01    2.8    288ms
  5   -7.926861635173       -6.83       -3.35    1.9    220ms
  6   -7.926861666984       -7.50       -3.73    1.8    216ms
  7   -7.926861680610       -7.87       -4.38    1.4    211ms
  8   -7.926861681819       -8.92       -4.99    2.2    258ms
  9   -7.926861681857      -10.42       -5.21    1.6    214ms
 10   -7.926861681872      -10.83       -6.01    1.5    210ms
 11   -7.926861681873      -12.01       -7.03    2.4    245ms
 12   -7.926861681873      -13.71       -7.35    3.0    272ms
 13   -7.926861681873   +  -15.05       -8.17    1.0    199ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921649404501                   -0.69    5.8         
  2   -7.926169304370       -2.34       -1.22    1.0    203ms
  3   -7.926841457270       -3.17       -2.37    1.8    271ms
  4   -7.926864882601       -4.63       -3.04    2.6    247ms
  5   -7.926865058830       -6.75       -3.42    1.9    214ms
  6   -7.926865082127       -7.63       -3.83    1.9    212ms
  7   -7.926865091138       -8.05       -4.26    1.1    217ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
